In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import numpy as np
import pandas as pd

import cv2
from skimage.feature import hog

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [ ]:
import os
print(os.listdir("/kaggle/input"))



In [ ]:
import os
print(os.listdir("/kaggle/input/final-data"))

In [ ]:
df_final = pd.read_csv("/kaggle/input/final-data/df_final.csv")
df_final.head()


In [ ]:
df_final["img_path"] = df_final["img_path"].str.replace(
    "/kaggle/working/resized_images",
    "/kaggle/input/final-data/resized_images",
    regex=False
)

df_final.head()


In [ ]:
X_images = np.array([load_image(p) for p in df_final["img_path"]])
y = df_final["label"].values

X_images.shape, y.shape


In [ ]:
#Safe check the correction of correct length of the label
len(df_final), len(y)


In [ ]:
#Extract HOG Features

from skimage.feature import hog

def extract_hog(img):
    gray = cv2.cvtColor((img*255).astype("uint8"), cv2.COLOR_RGB2GRAY)
    feat = hog(
        gray,
        orientations=9,
        pixels_per_cell=(16,16),
        cells_per_block=(2,2)
    )
    return feat

X_features = np.array([extract_hog(img) for img in X_images])
X_features.shape


In [ ]:
#Train/Test Split

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_features, 
    y, 
    test_size=0.2, 
    random_state=42,
    stratify=y
)

X_train.shape, X_test.shape


In [ ]:
#Scaling the features for kNN and Random Forest

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
#Train kNN Clasifier

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)

y_pred_knn = knn.predict(X_test_scaled)

print("KNN Accuracy:", accuracy_score(y_test, y_pred_knn))
print(classification_report(y_test, y_pred_knn))

#Accuracy after running:

# 0.7972 (≈79.7%)

# 2. Precision, Recall, F1 for Each Class

# Benign (0):
# Precision: 0.80, Recall: 0.99, F1: 0.89

# Malignant (1):
# Precision: 0.38, Recall: 0.03, F1: 0.06

In [ ]:
#Train Random Forest

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=200,
    random_state=42,
    n_jobs=-1
)

rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))

#Accurancy: 0.8018 (≈80.2%)